<a href="https://colab.research.google.com/github/jorgg3/Proyecto-3/blob/main/Ejercicio_22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Práctica 3: Ajuste por Mínimos Cuadrados y Factorización QR
**Problemas Computacionales**\
Alumno: Martínez de la Cruz José Jorge\
Profesor: César Carreón Otañez\
Ayudante:  Jesús Iván Coss Calderón

¿Cuál es la solución exacta al problema de mínimos cuadrados como función de $ε$ para el siguiente sistema?:
$$
\begin{bmatrix}
1 & 1 & 1 \\
\epsilon & 0 & 0 \\
0 & \epsilon & 0 \\
0 & 0 & \epsilon
\end{bmatrix}
\begin{pmatrix}
x_1 \\
x_2 \\
x_3
\end{pmatrix}
\approx
\begin{pmatrix}
1 \\
0 \\
0 \\
0
\end{pmatrix}
$$

Resolver el problema de mínimos cuadrados usando cada usando cada uno de los métodos listados. Para cada método variar el valor de $\epsilon$ tan pequeño como se pueda hasta obtener una solución exacta.
Probar explícitamente valores cercanos a
$$
ϵ ≈ \sqrt{ϵ_m} \, \, \, \, \, \, \, \ ϵ \approx ϵ_m
$$
Mostrar al menos 3 valores de $ϵ$ para los cuales se aplicaron las pruebas.

In [1]:
import numpy as np

## Método de Ecuaciones Normales

In [2]:
def SustDelante(L, b):
    """
    Resuelve un sistema de ecuaciones L * x = b, donde L es una matriz triangular inferior.

    Parámetros:
    L (numpy.ndarray): Matriz triangular inferior.
    b (numpy.ndarray): Vector columna de tamaño n, lado derecho del sistema.

    Retorna:
    x (numpy.ndarray): Vector solución del sistema L * x = b.
    """
    x = np.zeros_like(b)  # Crea un vector x de ceros del mismo tamaño que b
    n = L.shape[0]  # Obtiene el número de renglones de L
    for i in range(n):  # Recorre las filas de L
        sum = 0.0
        for j in range(i):  # Suma los productos de la parte ya resuelta de x
            sum += L[i, j] * x[j]
        # Resuelve para x[i]
        x[i] = (b[i] - sum) / L[i, i]

    return x  # Devuelve la solución

def SustAtras(U, y):
    """
    Resuelve un sistema de ecuaciones U * x = y, donde U es una matriz triangular superior.

    Parámetros:
    U (numpy.ndarray): Matriz triangular superior.
    y (numpy.ndarray): Vector columna de tamaño n, lado derecho del sistema.

    Retorna:
    x (numpy.ndarray): Vector solución del sistema U * x = y.
    """
    x = np.zeros_like(y)  # Crea un vector x de ceros del mismo tamaño que y
    n = U.shape[0]  # Obtiene el número de renglones de U
    x[n-1] = y[n-1] / U[n-1, n-1]  # Resuelve el último valor de x
    for i in range(n-2, -1, -1):  # Recorre las filas desde la penúltima hasta la primera
        sum = 0.0
        for j in range(i+1, n):  # Suma los productos de los valores ya resueltos de x
            sum += U[i, j] * x[j]
        # Resuelve para x[i]
        x[i] = (y[i] - sum) / U[i, i]

    return x  # Devuelve la solución



def Cholesky(A):
    """
    Realiza la descomposición de Cholesky de una matriz simétrica y definida positiva.

    La descomposición de Cholesky de una matriz A se expresa como:
    A = L * L^T
    donde:
        - A es la matriz de entrada (simétrica y definida positiva).
        - L es una matriz triangular inferior (con ceros sobre la diagonal).

    Parámetros:
    A (numpy.ndarray): Matriz cuadrada simétrica y definida positiva de tamaño n x n.

    Retorna:
    L (numpy.ndarray): Matriz triangular inferior de tamaño n x n tal que A = L * L^T.

    Excepciones:
    Si la matriz A no es simétrica o definida positiva, este algoritmo puede fallar
    o devolver resultados incorrectos.

    El algoritmo sigue el siguiente procedimiento:
    1. Inicializa una matriz L de ceros de las mismas dimensiones que A.
    2. Calcula los elementos de la matriz triangular inferior L usando la fórmula de Cholesky.
    3. La diagonal de L es calculada con la raíz cuadrada de los elementos de A.
    4. Los elementos fuera de la diagonal se calculan usando las sumas de productos previamente computadas.

    """
    # Determina el tamaño de la matriz A
    n = len(A)

    # Crea una matriz L de ceros del mismo tamaño que A
    L = np.zeros_like(A)

    # Recorre las filas y columnas de A para calcular los elementos de L
    for i in range(n):
        for j in range(i + 1):
            # Si estamos en la diagonal principal
            if i == j:
                # Calculamos la suma de los cuadrados de los elementos previos de la fila
                sum = 0.0
                for k in range(j):
                    sum += L[j][k] * L[j][k]
                # La raíz cuadrada del valor en A[j][j] menos la suma anterior da el valor de L[j][j]
                L[j][j] = np.sqrt(A[j][j] - sum)
            else:
                # Si estamos en la parte inferior de la matriz (por debajo de la diagonal)
                sum = 0.0
                for k in range(j):
                    sum += L[i][k] * L[j][k]
                # Usamos la fórmula para calcular los valores fuera de la diagonal
                L[i][j] = (A[i][j] - sum) / L[j][j]

    return L

In [3]:
def EcNormal(A, b):
    """
    Resuelve el sistema de ecuaciones lineales A * x = b utilizando el método de los mínimos cuadrados
    con la descomposición de Cholesky de la matriz A^T * A.

    Este método se utiliza cuando el sistema no tiene solución exacta o es sobredeterminado (más ecuaciones que incógnitas).
    En lugar de resolver el sistema directamente, se resuelve el sistema equivalente A^T * A * x = A^T * b,
    que tiene solución en el sentido de los mínimos cuadrados.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes del sistema de ecuaciones de tamaño m x n (m >= n).
    b (numpy.ndarray): Vector columna de tamaño m, que es el lado derecho del sistema de ecuaciones.

    Retorna:
    xSol (numpy.ndarray): El vector solución x, que es la solución de mínimos cuadrados al sistema A * x = b.

    Procedimiento:
    1. Calculamos el sistema equivalente A^T * A * x = A^T * b.
    2. Aplicamos la descomposición de Cholesky a la matriz A^T * A para obtener una matriz triangular inferior L.
    3. Resolvemos el sistema triangular inferior L * y = A^T * b mediante sustitución hacia adelante (SustDelante).
    4. Resolvemos el sistema triangular superior L^T * x = y mediante sustitución hacia atrás (SustAtras).
    """

    # Calculamos el sistema equivalente A^T * A * x = A^T * b
    AtA = A.T @ A  # Matriz generalizada (A^T * A)
    Atb = A.T @ b  # Vector del sistema equivalente (A^T * b)

    # Descomposición de Cholesky de A^T * A
    L = Cholesky(AtA)  # L es la matriz triangular inferior tal que A^T * A = L * L^T
    Lt = L.T  # La transpuesta de L, es la matriz triangular superior

    # Resolvemos el sistema L * y = A^T * b mediante sustitución hacia adelante
    ySol = SustDelante(L, Atb)

    # Resolvemos el sistema L^T * x = ySol mediante sustitución hacia atrás
    xSol = SustAtras(Lt, ySol)

    return xSol


Para $ϵ = 10 ^{-15}$

In [4]:
epsilon = 10**(-15)
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [5]:
EcNormal(A, b)

<ipython-input-2-1a14081ffdc9>:98: RuntimeWarning: invalid value encountered in scalar divide
  L[i][j] = (A[i][j] - sum) / L[j][j]
<ipython-input-2-1a14081ffdc9>:19: RuntimeWarning: invalid value encountered in scalar divide
  x[i] = (b[i] - sum) / L[i, i]


array([nan, nan, nan])

Con el epsilon escogido, nos encontramos con problemas computacionales de valores muy grandes o muy pequeños

In [6]:
epsilon = 10**(-10)
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [7]:
EcNormal(A, b)

<ipython-input-2-1a14081ffdc9>:98: RuntimeWarning: invalid value encountered in scalar divide
  L[i][j] = (A[i][j] - sum) / L[j][j]
<ipython-input-2-1a14081ffdc9>:19: RuntimeWarning: invalid value encountered in scalar divide
  x[i] = (b[i] - sum) / L[i, i]


array([nan, nan, nan])

Inclusive con $ϵ = 10^{-10}$, nos encontramos con estos problemas

In [8]:
epsilon = 10**(-16)
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [9]:
EcNormal(A, b)

<ipython-input-2-1a14081ffdc9>:98: RuntimeWarning: invalid value encountered in scalar divide
  L[i][j] = (A[i][j] - sum) / L[j][j]
<ipython-input-2-1a14081ffdc9>:19: RuntimeWarning: invalid value encountered in scalar divide
  x[i] = (b[i] - sum) / L[i, i]


array([nan, nan, nan])

¿Qué pasará para $ϵ = 0$?

In [10]:
epsilon = 0
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [11]:
EcNormal(A, b)

<ipython-input-2-1a14081ffdc9>:98: RuntimeWarning: invalid value encountered in scalar divide
  L[i][j] = (A[i][j] - sum) / L[j][j]


ValueError: cannot convert float NaN to integer

Hay errores computacionales en cuando $ϵ → 0$

## Factorización QR de Grand- Schmidht.




In [12]:
def QR(A):
    """
    Realiza la descomposición QR de una matriz A utilizando el método de Gram-Schmidt.

    Parámetros:
    A (numpy.ndarray): Matriz de entrada de tamaño (m x n), donde m es el número de filas y n el número de columnas.

    Retorna:
    Q (numpy.ndarray): Matriz ortogonal de tamaño (m x n), tal que Q^T * Q = I.
    R (numpy.ndarray): Matriz triangular superior de tamaño (n x n).

    El resultado satisface la relación A = Q * R, donde:
    - Q es una matriz ortogonal (Q^T * Q = I).
    - R es una matriz triangular superior.
    """

    # Inicialización de las matrices Q y R
    Q = np.empty_like(A)  # Matriz de salida Q, tendrá las mismas dimensiones que A
    R = np.zeros([A.shape[1], A.shape[1]])  # Matriz cuadrada R de tamaño n x n
    vi = np.zeros([A.shape[1]])  # Vector auxiliar vi

    # Iteración sobre las columnas de A
    for i in range(A.shape[1]):
        # Comienza con el vector de la columna i de A
        vi = A[:, i]

        # Proceso de ortogonalización de Gram-Schmidt
        for j in range(i):
            # Calculamos la proyección de vi sobre los vectores Q anteriores
            R[j, i] = np.dot(Q[:, j].T, vi)  # Proyección de vi sobre Q[:, j]
            # Restamos la proyección de vi para mantener la ortogonalidad
            vi = vi - R[j, i] * Q[:, j]

        # Normalizamos el vector vi para obtener un vector unitario
        R[i, i] = np.linalg.norm(vi, 2)  # Norma de vi, la longitud del vector
        Q[:, i] = vi / R[i, i]  # Normalizamos para obtener el i-ésimo vector ortonormal de Q

    # Retorna las matrices Q y R
    return Q, R

In [14]:
def SolQR(A, b):
    """
    Resuelve el sistema de ecuaciones lineales A * x = b utilizando la factorización QR de A.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes del sistema de ecuaciones lineales de tamaño (m x n),
                        donde m es el número de ecuaciones y n el número de incógnitas.
    b (numpy.ndarray): Vector de términos independientes del sistema de tamaño (m,).

    Retorna:
    x (numpy.ndarray): Solución del sistema de ecuaciones de tamaño (n,), tal que A * x = b.


    Después de realizar la descomposición QR, se resuelve el sistema Rx = Q^T * b. Este sistema triangular superior se resuelve usando
    la sustitución hacia atrás. La solución se obtiene en dos pasos:

    1. Calcular Q^T * b (esto se almacena en el vector bR).
    2. Resolver el sistema triangular superior Rx = bR usando la sustitución hacia atrás.

    """

    # Paso 1: Obtener la factorización QR de A
    Q, R = QR(A)

    # Paso 2: Calcular el vector bR = Q^T * b
    bR = Q.T @ b

    # Paso 3: Resolver el sistema triangular superior Rx = bR usando sustitución hacia atrás
    x = SustAtras(R, bR)

    return x


Nuevamente, para $ϵ = 10^{-10}$

In [18]:
epsilon = 10**(-10)
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [19]:
SolQR(A, b)

array([1., 0., 0.])

Nos indica que el vector $(1, 0, 0)$, es la solución al sistema

¿Qué pasa para $epsilon = 0?


In [23]:
epsilon = 0
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [24]:
SolQR(A, b)

<ipython-input-12-c665a8c1924b>:36: RuntimeWarning: invalid value encountered in divide
  Q[:, i] = vi / R[i, i]  # Normalizamos para obtener el i-ésimo vector ortonormal de Q
<ipython-input-12-c665a8c1924b>:36: RuntimeWarning: invalid value encountered in cast
  Q[:, i] = vi / R[i, i]  # Normalizamos para obtener el i-ésimo vector ortonormal de Q
<ipython-input-2-1a14081ffdc9>:36: RuntimeWarning: divide by zero encountered in scalar divide
  x[n-1] = y[n-1] / U[n-1, n-1]  # Resuelve el último valor de x


OverflowError: cannot convert float infinity to integer

Aquí también nos encontramos con problemas en cuanto $ϵ → 0$

##Factorización QR de HouseHolder

In [25]:
import numpy.linalg as LA

def Householder(A):
    """
    Realiza la descomposición QR de una matriz A utilizando el método de Householder.

    Parámetros:
    A (numpy.ndarray): Matriz de entrada de tamaño (m, n) que se desea descomponer.
                        La matriz A debe tener dimensiones m x n.

    Retorna:
    tuple: Una tupla (Q, R) donde:
        - Q (numpy.ndarray): Matriz ortogonal (m x m) tal que Q^T * Q = I.
        - R (numpy.ndarray): Matriz triangular superior (m x n) que contiene los coeficientes de la factorización QR.

    Descripción:
    El método de Householder se utiliza para descomponer una matriz A de tamaño m x n en una matriz ortogonal Q y una matriz triangular superior R tal que:

    A = Q * R

    El algoritmo se basa en la idea de aplicar reflexiones (matrices de Householder) para convertir las columnas de A en un sistema de vectores ortogonales.

    Procedimiento:
    1. Se iteran las columnas de la matriz A.
    2. Para cada columna, se genera un vector de Householder que "refleja" la columna actual hacia una dirección específica.
    3. Se actualizan las matrices \( Q \) y \( A \) en cada paso.
    4. La matriz \( Q \) se obtiene como el producto de las matrices de Householder acumuladas, mientras que la matriz \( A \) se va transformando en la matriz triangular superior \( R \).

    La función devuelve las matrices Q (ortogonal) y R (triangular superior) que cumplen la relación A = Q * R.

    Notas:
    - El vector xi es la copia de la columna de A a partir del elemento i-ésimo.
    - El vector ui es el vector de Householder, que es el eje de la reflexión.
    - El vector vi es normalizado para asegurar que la reflexión sea correcta.
    - Se actualiza la matriz \( A \) y la matriz acumulativa \( Q \) en cada iteración.
    """

    # Inicialización: Q es la matriz identidad y se va acumulando
    Q = np.eye(len(A))

    # Número de columnas de A
    n = A.shape[1]

    # Iterar sobre cada columna
    for i in range(n):
        # Inicializar xi, un vector con ceros
        xi = np.zeros(len(A))

        # Tomar el subvector de la columna de A a partir del i-ésimo índice
        xi[i:] = A[:, i][i:]

        # Calcular la norma de xi
        norm_x = LA.norm(xi)

        # Crear un vector unitario ei
        ei = np.zeros(len(A))
        ei[i] = 1.0

        # Generar el vector ui de Householder
        ui = xi + np.sign(A[i, i]) * norm_x * ei

        # Normalizar el vector ui para obtener vi
        vi = ui / LA.norm(ui)

        # Reshape de vi a un vector columna
        vi = vi.reshape(-1, 1)

        # Crear la matriz de Householder H
        H = np.eye(len(A)) - 2 * vi @ vi.T

        # Actualizar la matriz A
        A = H @ A

        # Actualizar la matriz Q
        Q = Q @ H

    return Q, A


In [38]:
def SustAtras(U, y):
    x = np.zeros_like(y)
    n = U.shape[0]  # cantidad de renglones de U
    x[n-1] = y[n-1] / U[n-1, n-1]
    for i in range(n-2, -1, -1):
        sum = 0.0
        for j in range(i+1, n):
            sum += U[i, j] * x[j]
        x[i] = (y[i] - sum) / U[i, i]

    return x

In [52]:
def SolQRH(A, b):
    """
    Resuelve el sistema de ecuaciones lineales A * x = b utilizando la factorización QR de A.

    Parámetros:
    A (numpy.ndarray): Matriz de coeficientes del sistema de ecuaciones lineales de tamaño (m x n),
                        donde m es el número de ecuaciones y n el número de incógnitas.
    b (numpy.ndarray): Vector de términos independientes del sistema de tamaño (m,).

    Retorna:
    x (numpy.ndarray): Solución del sistema de ecuaciones de tamaño (n,), tal que A * x = b.


    Después de realizar la descomposición QR, se resuelve el sistema Rx = Q^T * b. Este sistema triangular superior se resuelve usando
    la sustitución hacia atrás. La solución se obtiene en dos pasos:

    1. Calcular Q^T * b (esto se almacena en el vector bR).
    2. Resolver el sistema triangular superior Rx = bR usando la sustitución hacia atrás.

    """

    # Paso 1: Obtener la factorización QR de A
    Q, R = Householder(A)
    R = R[:-1, :]
    # Paso 2: Calcular el vector bR = Q^T * b
    bR = Q.T @ b
    bR = bR[:-1]
    # Paso 3: Resolver el sistema triangular superior Rx = bR usando sustitución hacia atrás
    x = SustAtras(R, bR)

    return x


Para $ϵ = 10^{-10}$, tenemos:

In [53]:
epsilon = 10**(-10)
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [54]:
SolQRH(A, b)

array([0.33333333, 0.33333333, 0.33333333])

En este caso, tenemos que la solución es $(\dfrac{1}{3}, \dfrac{1}{3}, \dfrac{1}{3})$

Veamos qué pasa cuando $ϵ = 10^{-16}$

In [55]:
epsilon = 10**(-16)
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [56]:
SolQRH(A, b)

array([0.33333333, 0.33333333, 0.33333333])

De la misma forma, tenemos problemas al tratar de encontrar la solución, ¿qué pasa entonces cuando queremos $ϵ >> 0$?

In [57]:
epsilon = 4
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [58]:
SolQRH(A, b)

array([0.05263158, 0.05263158, 0.05263158])

¿Y cuándo $ϵ = 0$?

In [61]:
epsilon = 0
A = np.array([[1, 1, 1], [epsilon, 0, 0], [0, epsilon, 0], [0, 0, epsilon]])
b = np.array([1, 0, 0, 0])

In [62]:
SolQRH(A, b)

<ipython-input-25-115fc343b9bd>:63: RuntimeWarning: invalid value encountered in divide
  vi = ui / LA.norm(ui)


array([nan, nan, nan])

Tenemos problemas cuando $\epsilon = 0$

##  Factorización QR de Givens